In [39]:
# load necessary libraries
import pandas
import numpy
import scipy
import statsmodels.api as stats
from sklearn import naive_bayes

print()
print("=" * 50)
print("=" * 50)
print("ML-Assignment 4-Question 2")
print("=" * 50)
print("=" * 50)
# Please answer the following questions based on your model.


ML-Assignment 4-Question 2


In [40]:
# define a function to visualize the percent of a particular target category by a nominal predictor
def row_with_column (rowVar, columnVar, show = 'ROW'):

    countTable = pandas.crosstab(index = rowVar, columns = columnVar, margins = False, dropna = True)
    print("Frequency Table: \n", countTable)
    print( )

    if (show == 'ROW' or show == 'BOTH'):
        rowFraction = countTable.div(countTable.sum(1), axis='index')
        print("Row Fraction Table: \n", rowFraction)
        print( )

    if (show == 'COLUMN' or show == 'BOTH'):
        columnFraction = countTable.div(countTable.sum(0), axis='columns')
        print("Column Fraction Table: \n", columnFraction)
        print( )

    return

In [41]:
# load data file
purchase_likelihood = pandas.read_csv('Purchase_Likelihood.csv', delimiter=',', usecols=['group_size', 'homeowner', 'married_couple', 'A'])
purchase_likelihood = purchase_likelihood.dropna()

# create category and interval variable list
cat_pred = ['group_size', 'homeowner', 'married_couple']
int_pred = cat_pred

In [42]:
print()
print("=" * 50)
print("ML-Assignment 4-Question 2-Section a)")
print("=" * 50)
# a)	(5 points) Show in a table the frequency counts and the Class Probabilities of the target variable.
freq = purchase_likelihood.groupby('A').size()
table = pandas.DataFrame(columns = ['count', 'class probability'])
table['count'] = freq
table['class probability'] = table['count']/purchase_likelihood.shape[0]
print(table)


ML-Assignment 4-Question 2-Section a)
    count  class probability
A                           
0  143691           0.215996
1  426067           0.640462
2   95491           0.143542


In [43]:
print()
print("=" * 50)
print("ML-Assignment 4-Question 2-Section b)")
print("=" * 50)
# b)	(5 points) Show the crosstabulation table of the target variable by the feature group_size.  The table contains the frequency counts.
row_with_column(purchase_likelihood['A'],purchase_likelihood['group_size'],'ROW')


ML-Assignment 4-Question 2-Section b)
Frequency Table: 
 group_size       1      2     3    4
A                                   
0           115460  25728  2282  221
1           329552  91065  5069  381
2            74293  19600  1505   93

Row Fraction Table: 
 group_size         1         2         3         4
A                                                 
0           0.803530  0.179051  0.015881  0.001538
1           0.773475  0.213734  0.011897  0.000894
2           0.778010  0.205255  0.015761  0.000974



In [44]:
print()
print("=" * 50)
print("ML-Assignment 4-Question 2-Section c)")
print("=" * 50)
# c)	(5 points) Show the crosstabulation table of the target variable by the feature homeowner.  The table contains the frequency counts.
row_with_column(purchase_likelihood['A'],purchase_likelihood['homeowner'],'ROW')


ML-Assignment 4-Question 2-Section c)
Frequency Table: 
 homeowner       0       1
A                        
0           78659   65032
1          183130  242937
2           46734   48757

Row Fraction Table: 
 homeowner         0         1
A                            
0          0.547418  0.452582
1          0.429815  0.570185
2          0.489407  0.510593



In [45]:
print()
print("=" * 50)
print("ML-Assignment 4-Question 2-Section d)")
print("=" * 50)
# d)	(5 points) Show the crosstabulation table of the target variable by the feature married_couple.  The table contains the frequency counts.
row_with_column(purchase_likelihood['A'],purchase_likelihood['married_couple'],'ROW')


ML-Assignment 4-Question 2-Section d)
Frequency Table: 
 married_couple       0      1
A                            
0               117110  26581
1               333272  92795
2                75310  20181

Row Fraction Table: 
 married_couple         0         1
A                                 
0               0.815013  0.184987
1               0.782206  0.217794
2               0.788661  0.211339



In [46]:
# define a function that performs the Chi-square test
def chi_square_test (x_cat, y_cat, debug = 'N'):

    obs_count = pandas.crosstab(index = x_cat, columns = y_cat, margins = False, dropna = True)
    c_total = obs_count.sum(axis = 1)
    r_total = obs_count.sum(axis = 0)
    n_total = numpy.sum(r_total)
    exp_count = numpy.outer(c_total, (r_total / n_total))

    if (debug == 'Y'):
        print('Observed Count:\n', obs_count)
        print('Column Total:\n', c_total)
        print('Row Total:\n', r_total)
        print('Overall Total:\n', n_total)
        print('Expected Count:\n', exp_count)
        print('\n')
       
    chi_sq_stat = ((obs_count - exp_count)**2 / exp_count).to_numpy().sum()
    chi_sq_df = (obs_count.shape[0] - 1.0) * (obs_count.shape[1] - 1.0)
    chi_sq_sig = scipy.stats.chi2.sf(chi_sq_stat, chi_sq_df)

    cramer_v = chi_sq_stat / n_total
    if (c_total.size > r_total.size):
        cramer_v = cramer_v / (r_total.size - 1.0)
    else:
        cramer_v = cramer_v / (c_total.size - 1.0)
    cramer_v = numpy.sqrt(cramer_v)

    return(chi_sq_stat, chi_sq_df, chi_sq_sig, cramer_v)

In [51]:
print()
print("=" * 50)
print("ML-Assignment 4-Question 2-Section e)")
print("=" * 50)
# e)	(10 points) Calculate the Cramer’s V statistics for the above three crosstabulations tables.  Based on these Cramer’s V statistics, which feature has the largest association with the target A?
test_result = pandas.DataFrame(index = cat_pred, columns = ['Test', 'Statistic', 'DF', 'Significance', 'Association', 'Measure'])
for pred in cat_pred:
    chi_sq_stat, chi_sq_df, chi_sq_sig, cramer_v = chi_square_test(purchase_likelihood[pred], purchase_likelihood['A'], debug = 'N')
    test_result.loc[pred] = ['Chi-square', chi_sq_stat, chi_sq_df, chi_sq_sig, 'Cramer''V', cramer_v]
rank_assoc = test_result.sort_values('Measure', axis = 0, ascending = False)
print(rank_assoc)


ML-Assignment 4-Question 2-Section e)
                      Test Statistic DF  Significance Association    Measure
homeowner       Chi-square   6270.49  2             0     CramerV  0.0970864
married_couple  Chi-square   699.285  2  1.41953e-152     CramerV  0.0324216
group_size      Chi-square   977.276  6  7.34301e-208     CramerV   0.027102


In [52]:
# train naive baues model
xTrain = purchase_likelihood[cat_pred].astype('category')
yTrain = purchase_likelihood['A'].astype('category')

_objNB = naive_bayes.MultinomialNB(alpha = 1.0e-10)
thisFit = _objNB.fit(xTrain, yTrain)

In [54]:
print()
print("=" * 50)
print("ML-Assignment 4-Question 2-Section g)")
print("=" * 50)
# g)	(10 points) For each of the sixteen possible value combinations of the three features, calculate the predicted probabilities for A = 0, 1, 2 based on the Naïve Bayes model.  List your answers in a table with proper labelling.
# Create the all possible combinations of the features' values
gs_d = [1,2,3,4]
ho_d = [0,1]
mc_d = [0,1]
A_d = [0,1,2]
final_data = []
for gsd in gs_d:
    for hod in ho_d:
        for mcd in mc_d:
            data = [gsd,hod,mcd]
            final_data = final_data + [data]
x_test = pandas.DataFrame(final_data, columns=['group_size','homeowner','married_couple'])
x_test = x_test[cat_pred].astype('category')
# predict probabities for all combinations
y_test_pred_prob = pandas.DataFrame(_objNB.predict_proba(x_test), columns = ['p_a_0', 'p_a_1','p_a_2'])
y_test_score = pandas.concat([x_test, y_test_pred_prob], axis = 1)                                          
print(y_test_score)


ML-Assignment 4-Question 2-Section g)
   group_size homeowner married_couple     p_a_0     p_a_1     p_a_2
0           1         0              0  0.227037  0.627593  0.145370
1           1         0              1  0.214391  0.637467  0.148142
2           1         1              0  0.205588  0.654128  0.140284
3           1         1              1  0.193842  0.663414  0.142744
4           2         0              0  0.238441  0.614462  0.147097
5           2         0              1  0.225342  0.624635  0.150024
6           2         1              0  0.216281  0.641528  0.142192
7           2         1              1  0.204079  0.651128  0.144794
8           3         0              0  0.250201  0.601084  0.148715
9           3         0              1  0.236653  0.611546  0.151801
10          3         1              0  0.227342  0.628652  0.144006
11          3         1              1  0.214684  0.638559  0.146756
12          4         0              0  0.262308  0.587475  0.15

In [55]:
print()
print("=" * 50)
print("ML-Assignment 4-Question 2-Section g)")
print("=" * 50)
# h)	(5 points) Based on your model, what values of group_size, homeowner, and married_couple will maximize the odds value Prob(A=1) / Prob(A = 0)?  What is that maximum odd value?
y_test_score['odd value(p_a_1/p_a_0)'] = y_test_score['p_a_1'] / y_test_score['p_a_0']
print(y_test_score[['group_size','homeowner','married_couple','odd value(p_a_1/p_a_0)']])
print(y_test_score.loc[y_test_score['odd value(p_a_1/p_a_0)'].idxmax()])


ML-Assignment 4-Question 2-Section g)
   group_size homeowner married_couple  odd value(p_a_1/p_a_0)
0           1         0              0                2.764273
1           1         0              1                2.973389
2           1         1              0                3.181743
3           1         1              1                3.422441
4           2         0              0                2.576994
5           2         0              1                2.771943
6           2         1              0                2.966181
7           2         1              1                3.190572
8           3         0              0                2.402403
9           3         0              1                2.584145
10          3         1              0                2.765223
11          3         1              1                2.974412
12          4         0              0                2.239641
13          4         0              1                2.409070
14          4   